# Introducción
---
**Nombre:** Eduardo González Gutiérrez

**alu:** alu0101461588

---
En este cuaderno Jupyter se presenta el desarrollo de dos modelos matemáticos relacionados con problemas de asignación tridimensional, implementados en Julia utilizando el paquete JuMP para el modelado y GLPK como solucionador. Los modelos planteados son:

* **Problema de asignación tridimensional planar:** Un modelo en el que se busca asignar elementos en un espacio 3D minimizando el coste total, sujeto a restricciones que garantizan que cada fila, columna y capa sea cubierta exactamente una vez.

* **Problema de asignación tridimensional axial:** Similar al problema planar, pero con un enfoque diferente en la estructura y las restricciones.

En cada modelo se incluyen:

* Definición de variables, función objetivo y restricciones.
* Resolución utilizando el solucionador GLPK.
* Análisis y presentación de los resultados obtenidos.

Este cuaderno representa un ejercicio práctico para aplicar los conocimientos adquiridos en la asignatura Ingeniería Logística, trabajando con problemas de optimización de asignación.





# Instalar Julia y Kernel de Julia

In [ ]:
# Descargar e instalar Julia
!wget https://julialang-s3.julialang.org/bin/linux/x64/1.9/julia-1.9.3-linux-x86_64.tar.gz -O julia.tar.gz
!tar -xvzf julia.tar.gz
!mv julia-1.9.3 /usr/local/julia
!ln -s /usr/local/julia/bin/julia /usr/local/bin/julia


# Instalar paquetes de Julia
!julia -e 'using Pkg; Pkg.add("IJulia"); Pkg.add("JuMP"); Pkg.add("GLPK"); Pkg.add("Random")'


# Configurar el kernel de Julia en Colab
import os
os.system("julia -e 'using IJulia; IJulia.installkernel(\"Julia\")'")


# Modelo Planar: Asignación de Transporte de Material entre Almacenes y Hospitales

##Introducción al Modelo Planar

El modelo planar es un tipo de modelo matemático utilizado en problemas de asignación en los que se deben distribuir recursos entre diferentes ubicaciones, minimizando un coste total asociado a dicha distribución. En este caso, el modelo planar se utiliza para resolver un problema de asignación entre tres elementos: hospitales, transportistas y almacenes.

El modelo se organiza en un espacio tridimensional, donde cada dimensión representa uno de los elementos involucrados en la asignación. Los hospitales necesitan recibir material desde los almacenes, y para ello, los transportistas son los encargados de realizar las entregas. El objetivo es determinar qué transportista, asignado a un hospital y un almacén, debe realizar cada entrega de manera que el coste total del transporte se minimice.

## Enunciado del Problema
En este problema de asignación, se tiene un conjunto de hospitales, transportistas y almacenes. El objetivo es asignar a cada hospital un transportista y un almacén de manera eficiente, de forma que se minimice el coste total de transporte.

Datos del problema:
* H: Número de hospitales.
* T: Número de transportistas.
* A: Número de almacenes.

Se cuenta con una **matriz de costes** que representa el coste de transportar material desde un almacén hasta un hospital por medio de un transportista.

El problema está sujeto a las siguientes restricciones:

1. Cada hospital debe recibir material de un único transportista y un único almacén.
2. Cada transportista puede realizar como máximo una entrega.
3. Cada almacén puede enviar material a un máximo de un hospital.

El objetivo es encontrar la asignación de transportistas a hospitales y almacenes que minimice el coste total de transporte, respetando las restricciones mencionadas.


In [2]:
# Instalación de paquetes necesarios
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

# Uso de paquetes
using JuMP, GLPK, Random

# Datos del problema
H = 4         # Número de hospitales
T = H         # Número de transportistas
A = T         # Número de almacenes
Random.seed!(1234)  # Semilla para generar datos reproducibles

# Coste de transportar material desde un almacén a un hospital por un transportista
Coste = rand(30:80, H, T, A)

# Creación del modelo
model = Model(GLPK.Optimizer)
set_silent(model)

# Definir variables de decisión
@variable(model, x[1:H, 1:T, 1:A] >= 0, binary=true)

# Definir la función objetivo (minimizar el coste total de transporte)
@objective(model, Min, sum(Coste[h, t, a] * x[h, t, a] for h in 1:H, t in 1:T, a in 1:A))

# Restricciones
@constraint(model, cada_hospital[h=1:H], sum(x[h, :, :]) == 1) # Cada hospital recibe material de un almacén y un transportista
@constraint(model, cada_transportista[t=1:T], sum(x[:, t, :]) <= 1) # Cada transportista realiza una entrega
@constraint(model, capacidad_almacen[a=1:A], sum(x[:, :, a]) <= 1) # Cada almacén envía a un máximo de un hospital

# Optimización del modelo
optimize!(model)

# Resultado de la optimización
if termination_status(model) == MOI.OPTIMAL
    println("Solución Óptima Encontrada:")
    println("Coste total: ", objective_value(model), "€.")
    println()

    for h in 1:H
        for t in 1:T
            for a in 1:A
                if value(x[h, t, a]) > 0.5
                    println("El transportista $t lleva material del almacén $a al hospital $h con un coste de ", Coste[h, t, a], "€.")
                end
            end
        end
    end
else
    println("No se encontró una solución óptima.")
end


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


Solución Óptima Encontrada:
Coste total: 157.0€.

El transportista 4 lleva material del almacén 1 al hospital 1 con un coste de 30€.
El transportista 3 lleva material del almacén 2 al hospital 2 con un coste de 41€.
El transportista 1 lleva material del almacén 4 al hospital 3 con un coste de 52€.
El transportista 2 lleva material del almacén 3 al hospital 4 con un coste de 34€.


# **Modelo Axial: Asignación de Tareas a Trabajadores en un Proyecto**
##Introducción al Modelo Axial
El modelo axial es útil para resolver problemas de asignación en los que se deben asignar recursos entre varios grupos en un espacio tridimensional. Este modelo es adecuado cuando se tienen diferentes tareas, trabajadores y equipos que deben ser asignados de manera eficiente, minimizando un coste total asociado a la asignación.

En este caso, el modelo axial se utiliza para asignar tareas a trabajadores y equipos en un proyecto, donde cada tarea debe ser asignada a un trabajador y un equipo específicos. El objetivo es minimizar el coste total de la asignación, que depende de la tarea, el trabajador y el equipo involucrado.

## Enunciado del Problema
En este problema de asignación, se tiene un conjunto de tareas, trabajadores y equipos, y se desea asignar cada tarea a un trabajador y un equipo de manera eficiente, minimizando el coste total de la asignación.

Datos del problema:
1. T: Número de tareas a realizar.
2. W: Número de trabajadores disponibles.
3. E: Número de equipos disponibles.

Se cuenta con una **matriz de costes** que representa el coste de asignar una tarea específica a un trabajador y a un equipo determinados.

El problema está sujeto a las siguientes restricciones:

1. Cada tarea debe ser asignada exactamente a un trabajador y un equipo.
2. Cada trabajador puede realizar como máximo una tarea.
3. Cada equipo puede realizar como máximo una tarea.

El objetivo es encontrar la asignación de tareas a trabajadores y equipos que minimice el coste total, respetando las restricciones mencionadas.

In [4]:
# Instalación de paquetes necesarios
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

# Uso de paquetes
using JuMP, GLPK, Random

# Datos del problema
T = 4  # Número de tareas
W = 4  # Número de trabajadores
E = 4  # Número de equipos
Random.seed!(1234)  # Semilla para generar datos reproducibles

# Matriz de costes aleatorios (ajusta según el problema real)
coste = rand(10:50, T, W, E)

# Crear el modelo
model_axial = Model(GLPK.Optimizer)

# Definir variables de decisión: asignación 3D (tarea, trabajador, equipo)
@variable(model_axial, asignacion_axial[t=1:T, w=1:W, e=1:E] >= 0, binary=true)

# Función objetivo: minimizar el coste total
@objective(model_axial, Min, 
    sum(coste[t, w, e] * asignacion_axial[t, w, e] for t in 1:T, w in 1:W, e in 1:E))

# Restricciones

# Restricción 1: Cada tarea debe ser asignada exactamente a un trabajador y un equipo
@constraint(model_axial, [t=1:T], sum(asignacion_axial[t, w, e] for w in 1:W, e in 1:E) == 1)

# Restricción 2: Cada trabajador realiza como máximo una tarea
@constraint(model_axial, [w=1:W], sum(asignacion_axial[t, w, e] for t in 1:T, e in 1:E) <= 1)

# Restricción 3: Cada equipo realiza como máximo una tarea
@constraint(model_axial, [e=1:E], sum(asignacion_axial[t, w, e] for t in 1:T, w in 1:W) <= 1)

# Resolver el modelo
optimize!(model_axial)

# Resultados
println("\nModelo Axial:")
println("Estado de optimización: ", termination_status(model_axial))
println("Costo óptimo: ", objective_value(model_axial))
println("Variables no nulas:")

# Mostrar las asignaciones no nulas
for t in 1:T, w in 1:W, e in 1:E
    if value(asignacion_axial[t, w, e]) > 0.5  # Solo mostrar asignaciones no nulas
        println("asignacion_axial[$t, $w, $e] = ", value(asignacion_axial[t, w, e]))
    end
end


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`



Modelo Axial:
Estado de optimización: OPTIMAL
Costo óptimo: 70.0
Variables no nulas:
asignacion_axial[1, 4, 1] = 1.0
asignacion_axial[2, 3, 2] = 1.0
asignacion_axial[3, 1, 4] = 1.0
asignacion_axial[4, 2, 3] = 1.0
